In [ ]:
import numpy as np
import pandas as pd

# ==============================
# Monte Carlo：根据 GARCH 年化波动率生成 FX 场景
# ==============================

# vol_cny = annualized_vol  # 在 USDCNY 那段 GARCH 结束时
# vol_mxn = annualized_vol  # 在 USDMXN 那段 GARCH 结束时

print("Using GARCH-based annualized volatilities:")
print(f"  USDCNY: {vol_cny:.2%}")
print(f"  USDMXN: {vol_mxn:.2%}\n")

# 1. 设定模拟次数
n_sims = 20000
rng = np.random.default_rng(42)   # 固定随机种子，便于复现

# 2. 把年化波动率当成 1 年 horizon 的 sigma
sigma_cny = vol_cny
sigma_mxn = vol_mxn

# 3. 抽样 FX log-return shocks
#    ΔFX ~ N(0, sigma^2)
shocks_cny = rng.normal(loc=0.0, scale=sigma_cny, size=n_sims)
shocks_mxn = rng.normal(loc=0.0, scale=sigma_mxn, size=n_sims)

# 4. 将 shock 转成“成本乘子”：成本 ≈ 基础成本 × exp(ΔFX)
#    例如 multiplier = 1.10 表示成本 +10%
mult_cny = np.exp(shocks_cny)
mult_mxn = np.exp(shocks_mxn)

# 5. 组装成一个场景表：每行 = 一个 Monte Carlo 场景
scenarios = pd.DataFrame({
    "scenario_id": np.arange(n_sims),
    "fx_shock_cny": shocks_cny,
    "fx_shock_mxn": shocks_mxn,
    "cost_multiplier_cn": mult_cny,
    "cost_multiplier_mx": mult_mxn,
})

print("Head of FX scenarios:")
print(scenarios.head())

# 6. 保存到 CSV，后面在 risk_parameters / MILP 中调用
scenarios.to_csv("fx_scenarios_from_garch.csv", index=False)
print("\nSaved FX scenarios to fx_scenarios_from_garch.csv")


Using GARCH-based annualized volatilities:
  USDCNY: 4.48%
  USDMXN: 12.94%

Head of FX scenarios:
   scenario_id  fx_shock_cny  fx_shock_mxn  cost_multiplier_cn  \
0            0      0.013641     -0.214581            1.013735   
1            1     -0.046556      0.115327            0.954511   
2            2      0.033595     -0.007510            1.034166   
3            3      0.042106     -0.077632            1.043005   
4            4     -0.087341     -0.032457            0.916365   

   cost_multiplier_mx  
0            0.806880  
1            1.122241  
2            0.992518  
3            0.925305  
4            0.968064  

Saved FX scenarios to fx_scenarios_from_garch.csv


In [5]:
# ==============================
# Monte Carlo：使用 GARCH σ 路径
# ==============================
n_sims = 20000
T = len(sigma_cny)  # 应该 =252
rng = np.random.default_rng(42)

print("Using GARCH-based daily volatility paths for Monte Carlo:")
print(f"  USDCNY annualized vol: {vol_cny:.2%}")
print(f"  USDMXN annualized vol: {vol_mxn:.2%}\n")

# 1) 生成标准正态噪声
eps_cny = rng.standard_normal((n_sims, T))
eps_mxn = rng.standard_normal((n_sims, T))

# 2) 根据每日 sigma 生成每日 log-return
daily_ret_cny = eps_cny * sigma_cny      # 自动广播 (20000×252)
daily_ret_mxn = eps_mxn * sigma_mxn

# 3) 一年总 log-return
annual_logret_cny = daily_ret_cny.sum(axis=1)
annual_logret_mxn = daily_ret_mxn.sum(axis=1)

# 4) 成本乘子 = exp(ΔFX)
mult_cny = np.exp(annual_logret_cny)
mult_mxn = np.exp(annual_logret_mxn)

# 5) 场景表
scenarios = pd.DataFrame({
    "scenario_id": np.arange(n_sims),
    "fx_annual_logret_cny": annual_logret_cny,
    "fx_annual_logret_mxn": annual_logret_mxn,
    "cost_multiplier_cny": mult_cny,
    "cost_multiplier_mxn": mult_mxn,
})

print("Head of scenario table:")
print(scenarios.head())

# 6) 导出
scenarios.to_csv("fx_scenarios_from_garch_paths.csv", index=False)
print("\nSaved: fx_scenarios_from_garch_paths.csv")


Using GARCH-based daily volatility paths for Monte Carlo:
  USDCNY annualized vol: 6.38%
  USDMXN annualized vol: 12.24%

Head of scenario table:
   scenario_id  fx_annual_logret_cny  fx_annual_logret_mxn  \
0            0             -0.056810             -0.074275   
1            1              0.007566              0.055839   
2            2             -0.099665              0.114832   
3            3              0.000879              0.044008   
4            4             -0.024072             -0.077381   

   cost_multiplier_cny  cost_multiplier_mxn  
0             0.944774             0.928417  
1             1.007594             1.057428  
2             0.905141             1.121685  
3             1.000880             1.044991  
4             0.976215             0.925537  

Saved: fx_scenarios_from_garch_paths.csv
